In [1]:
# Carregando as bibliotecas

import pandas as pd
import numpy as np
import yfinance as yf

from plotly.subplots import make_subplots
import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default = 'png'  ## comentar se quiser grafico interativo

from math import sqrt
from sklearn.cluster import KMeans

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

In [2]:
tickers = ["^BVSP", "BRL=X", "BOVA11.SA", "PETR4.SA", "VALE", "ITUB4.SA"
           , "CSNA3.SA", "BBAS3.SA", "WEGE3.SA", "B3SA3.SA" ,  "JBSS3.SA"
           , "ITSA4.SA", "SBSP3.SA", "CCRO3.SA", "UGPA3.SA"
           , "MULT3.SA", "MGLU3.SA", "EQTL3.SA", "LREN3.SA"
           ,"ABEV3.SA", "SUZB3.SA", "^GSPC", "BTC-USD", "ETH-USD"
           , "GBPUSD=X", "EURUSD=X", "^VIX"]

In [3]:
data =  yf.download(tickers[0], start="2022-01-01")["Adj Close"]
base_ativos = pd.DataFrame(data)
base_ativos.columns = [tickers[0]]


for ticker in tickers[1:]:
    try:
        cotacoes = yf.download(ticker, start="2022-01-01")["Adj Close"]
        cotacoes = pd.DataFrame(cotacoes)
        cotacoes.columns = [ticker]
        # ativos.append(cotacoes)/
    except:
        pass
    base_ativos = pd.concat([base_ativos,cotacoes], axis = 1)
base_ativos.sort_index(inplace = True)
base_ativos.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,^BVSP,BRL=X,BOVA11.SA,PETR4.SA,VALE,ITUB4.SA,CSNA3.SA,BBAS3.SA,WEGE3.SA,B3SA3.SA,...,EQTL3.SA,LREN3.SA,ABEV3.SA,SUZB3.SA,^GSPC,BTC-USD,ETH-USD,GBPUSD=X,EURUSD=X,^VIX
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,47686.812500,3769.697021,NaN,NaN,NaN
2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,47345.218750,3829.564941,NaN,NaN,NaN
2022-01-03,103922.0,5.5693,99.930000,12.735184,11.524237,18.931469,18.081091,11.989799,30.654587,9.822482,...,20.923018,21.289326,13.839315,54.899971,4796.560059,46458.117188,3761.380371,1.352228,1.137346,16.60
2022-01-04,103514.0,5.6812,99.570000,12.783340,11.307270,19.468349,17.808142,12.002282,30.501408,10.033223,...,20.733414,21.031046,13.821261,56.097652,4793.540039,45897.574219,3794.056641,1.348327,1.130224,16.91
2022-01-05,101006.0,5.6762,97.099998,12.288645,11.399062,19.098698,17.430025,11.802589,28.893049,9.822482,...,20.458485,20.007168,13.550432,55.461960,4700.580078,43569.003906,3550.386963,1.353143,1.128363,19.73


In [4]:
# Vamos criar e agrupar as informações com base anual média

retornos = pd.DataFrame(base_ativos.pct_change().mean()*252)
retornos.columns = ["Retornos"]
retornos["Volatilidade"] = base_ativos.pct_change().std()*sqrt(252)

# Ajusta os dados para o k-means

x = retornos[["Retornos", "Volatilidade"]]
min_clusters = 2
max_clusters = 10
inertias = np.zeros(shape = (max_clusters - min_clusters + 1,))

for i in range(min_clusters, max_clusters + 1):
    kmeans = KMeans(n_clusters = i, random_state = 42)
    kmeans.fit(x)
    inertias[i - min_clusters] = kmeans.inertia_

In [5]:
# Gráfico do método do cotovelo

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = list(range(2,10)), y = inertias
                         , name = "Inertia", line = dict(color = "blue"))
              , row = 1, col = 1)


fig.update_layout(height = 400, width = 600
                  , title_text = "Determinação do # de clusters - Método do cotovelo"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Número de Clusters"
                  , yaxis_title = "Inertia"
                  , legend_title = "Study objects"
                  , font = dict(size = 15, color = "Black")
                 )
fig.update_layout(hovermode = "x")

fig.show()

In [6]:
# Ajuste do K-Means

kmodel = KMeans(n_clusters = 5, random_state = 42)
clusters = kmodel.fit(x)

In [7]:
# Acessando os clusters

clusters.labels_

array([4, 4, 4, 2, 0, 2, 0, 2, 0, 0, 0, 4, 2, 0, 2, 0, 3, 4, 0, 4, 0, 4,
       1, 1, 4, 4, 1], dtype=int32)

In [8]:
retornos["Clusters"] = clusters.labels_

In [9]:
retornos

,Retornos,Volatilidade,Clusters
^BVSP,0.054454,0.149537,4
BRL=X,0.000344,0.122297,4
BOVA11.SA,0.058624,0.155950,4
PETR4.SA,0.339242,0.294116,2
VALE,0.042003,0.313382,0
ITUB4.SA,0.165009,0.197413,2
CSNA3.SA,-0.029628,0.376370,0
BBAS3.SA,0.247748,0.223328,2
WEGE3.SA,0.112164,0.247150,0
B3SA3.SA,0.064644,0.313224,0


In [10]:
# Avaliação dos resultados - 2 métodos de construir o gráfico
# Método 1

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = retornos.loc[retornos["Clusters"] == 0, "Volatilidade"]
                         , y = retornos.loc[retornos["Clusters"] == 0, "Retornos"]
                         , name = "Grupo 1", mode = "markers"
                         , text = retornos.loc[retornos["Clusters"] == 0].index
                         , marker = dict(size = 16, color = "red"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = retornos.loc[retornos["Clusters"] == 1, "Volatilidade"]
                         , y = retornos.loc[retornos["Clusters"] == 1, "Retornos"]
                         , name = "Grupo 2", mode = "markers"
                         , text = retornos.loc[retornos["Clusters"] == 1].index
                         , marker = dict(size = 16, color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = retornos.loc[retornos["Clusters"] == 2, "Volatilidade"]
                         , y = retornos.loc[retornos["Clusters"] == 2, "Retornos"]
                         , name = "Grupo 3", mode = "markers"
                         , text = retornos.loc[retornos["Clusters"] == 2].index
                         , marker = dict(size = 16, color = "black"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = retornos.loc[retornos["Clusters"] == 3, "Volatilidade"]
                         , y = retornos.loc[retornos["Clusters"] == 3, "Retornos"]
                         , name = "Grupo 4", mode = "markers"
                         , text = retornos.loc[retornos["Clusters"] == 3].index
                         , marker = dict(size = 16, color = "purple"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = retornos.loc[retornos["Clusters"] == 4, "Volatilidade"]
                         , y = retornos.loc[retornos["Clusters"] == 4, "Retornos"]
                         , name = "Grupo 5", mode = "markers"
                         , text = retornos.loc[retornos["Clusters"] == 4].index
                         , marker = dict(size = 16, color = "darkgreen"))
              , row = 1, col = 1)


fig.update_layout(height = 400, width = 600
                  , title_text = "K-Means para seleção de ativos"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Retornos"
                  , yaxis_title = "Volatidade"
                  , showlegend = True
                  , legend_title = "Alocação"
                  , font = dict(size = 13, color = "Black")
                 )
fig.update_layout(hovermode = "closest")

fig.show()

In [11]:
# Avaliação dos resultados - 2 métodos de construir o gráfico
# Método 2

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = retornos["Volatilidade"], y = retornos["Retornos"]
                         , name = "", mode = "markers"
                         , text = retornos.index
                         , marker = dict(size = 16, color = retornos["Clusters"]))
              , row = 1, col = 1)


fig.update_layout(height = 400, width = 600
                  , title_text = "K-Means para seleção de ativos"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Retornos"
                  , yaxis_title = "Volatidade"
                  , showlegend = False
                  , legend_title = "Alocação"
                  , font = dict(size = 13, color = "Black")
                 )
fig.update_layout(hovermode = "closest")

fig.show()